# *Template*: Probing the coordinates of a model grid

---

## Overview
The coordinate system that is computationally practical for model simulations is not always intuitive for the slicing and dicing required by analysis. 

1. Intuitive notion of spatial distributions in latitude-longitude coordinates 
1. Reality of model grids


---

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import intake
import numpy as np
import xarray as xr

import cftime

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter


import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil

import pyleoclim as pyleo

## Load Data

In [ ]:
file_path = #path
ds = xr.open_dataset(file_path, use_cftime=True)
ds

## Unit Conversions
Sometimes the units shipped with a dataset are not convenient. 

In [ ]:
orig_dim = #string
new_dim = #string

unit_conversion = #float
ds=ds.assign(      =ds[orig_dim] * unit_conversion) #fill in the blank with the new_dim, but without the quotes denoting string
ds = ds.swap_dims({orig_dim:new_dim}) 

### Example 1: depth

In [ ]:
orig_dim = 'z_t' #string
new_dim = 'z_t_m'#string

depth_unit_conversion = 1/100
ds = ds.assign(z_t_m=ds[orig_dim] * depth_unit_conversion) #note z_t_m=ds syntax
ds = ds.swap_dims({orig_dim:new_dim})

### Example 2: longitude

In [ ]:
orig_dim = 'lon' #string
new_dim = 'lon_360'#string

ds = ds.assign(lon_360=ds[orig_dim] %360) #note lon_360=ds syntax
ds = ds.swap_dims({orig_dim:new_dim})

## Shape of Coordinates and Dimensions

#### Coordinates

In [ ]:
ds.coords

In [ ]:
lat_coord = #string
lon_coord = #string

In [ ]:
'shape of latitude coordinate: {}'.format(ds.coords[lat_coord].shape)

In [ ]:
'shape of longitude coordinate: {}'.format(ds.coords[lon_coord].shape)

#### Dimensions

In [ ]:
ds.dims

If the latitude and longitude coordinates are vectors, and there are related dimensions such that:

In [ ]:
ds.dims[lat_dim]==ds.coords[lat_coord].shape[0]

Then working with this dataset should be spatially straightforward. You'll have full use of `.sel` for querying and will find it most straightforward to use `contourf` for distribution plots. 

## Data with latitude mesh and longitude mesh
If the latitude and longitude coordinates are a 2D array, try the plot below to visualize the locations of the data.

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()

ax.scatter(ds.coords[lon_coord], ds.coords[lat_coord], s=.51);
ax.set_xlabel(r'Longitude ($^{\circ}$E)')
ax.set_ylabel(r'Latitude ($^{\circ}$N)')


In order to plot a map, pick a variable of interest

In [272]:
var = #string
time = #float (if applicable)
depth_dim = #string
depth = #float (if applicable)

In [274]:
snapshot_data = ds.sel(**{depth_dim:depth, 'time':time}, method='nearest')
snapshot_data = snapshot_data[var].squeeze()
snapshot_data = snapshot_data.compute()

In [ ]:
# plt.figure(figsize=(9,5));
fig = plt.figure(figsize=(8, 4))

# add subplot with specified map projection and coastlines (GeoAxes)
ax2 = fig.add_subplot(projection=ccrs.Robinson(central_longitude=0))

snapshot_data.plot.pcolormesh(ax=ax2, 
                                   transform=ccrs.PlateCarree(), 
                                   x=lon_coord, y=lat_coord, 
                                   levels=ax2_levels, 
                                  cmap='viridis', 
                                  add_colorbar=True);

ax2.coastlines(linewidth=.5)
ax2.add_feature(cfeature.LAND, zorder=14)
